# AlphaMissense Score Statistical Testing

This notebook performs statistical testing on AlphaMissense scores, similar to the dN/dS analysis in analysis.ipynb. We'll calculate p-values, q-values, and identify significantly altered genes based on AlphaMissense scores.

In [20]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from statsmodels.stats.multitest import multipletests

## Load Gene Results with AlphaMissense Scores

First, let's load the gene results that include both expected and observed AlphaMissense scores.

In [21]:
# Load the gene results with observed DN scores
results_file = "/Volumes/T7_Shield/Lab/T7_UCSD/github_repos/BENG285_team3/project_2/alphamissense/gene_results_with_observed_dn_scores.csv"
gene_results = pd.read_csv(results_file)
print(f"Loaded data for {len(gene_results)} genes")

# Check if we have the expected columns
required_columns = ['gene_name', 'observed_mis_am_score', 'exp_mis_scores', 'observed_dn_score', 'exp_dn_score']
missing_columns = [col for col in required_columns if col not in gene_results.columns]
if missing_columns:
    print(f"Warning: Missing required columns: {missing_columns}")
else:
    print("All required columns are present")

# Display the first few rows
gene_results[required_columns].head()

Loaded data for 15520 genes
All required columns are present


,gene_name,observed_mis_am_score,exp_mis_scores,observed_dn_score,exp_dn_score
0,KRAS,111.0805,1725.7586,112.0805,1948.7586
1,TP53,82.0906,1120.5145,107.0906,2090.5145
2,KEAP1,41.6035,2161.5357,49.6035,2969.5357
3,EGFR,39.7970,7976.3108,40.7970,10257.3108
4,STK11,19.2695,3284.3718,33.2695,3616.3718


In [23]:
import numpy as np
import pandas as pd
from statsmodels.stats.rates import test_poisson_2indep
from statsmodels.stats.multitest import multipletests

def test_am_rate_ratio(gene_df, alpha=0.1):
    """
    Perform Poisson rate ratio test comparing observed and expected AM scores normalized by synonymous counts.
    
    Parameters:
    -----------
    gene_df: pd.DataFrame
        DataFrame containing 'observed_mis_am_score', 'n_syn', 'exp_mis_scores', 'exp_syn_counts'.
    alpha: float
        Significance level for multiple testing correction.
        
    Returns:
    --------
    pd.DataFrame
        DataFrame with additional columns: 'am_rate_p_value', 'am_rate_q_value', 'am_rate_significant'.
    """
    results = gene_df.copy()
    p_values = []

    for idx, row in results.iterrows():
        obs_score = row['observed_mis_am_score']
        obs_syn = row['n_syn']
        exp_score = row['exp_mis_scores']
        exp_syn = row['exp_syn_counts']

        # Ensure valid inputs
        if all(pd.notnull([obs_score, obs_syn, exp_score, exp_syn])) and all(np.array([obs_score, obs_syn, exp_score, exp_syn]) > 0):
            # Perform one-sided test: H0: rate_obs <= rate_exp vs H1: rate_obs > rate_exp
            stat, p_value = test_poisson_2indep(count1=obs_score, exposure1=obs_syn,
                                                count2=exp_score, exposure2=exp_syn,
                                                method='score', compare='ratio',
                                                alternative='larger')
        else:
            p_value = np.nan

        p_values.append(p_value)

    results['am_rate_p_value'] = p_values

    # Multiple testing correction
    valid_pvals = results['am_rate_p_value'].notna()
    if valid_pvals.any():
        _, q_values, _, _ = multipletests(results.loc[valid_pvals, 'am_rate_p_value'], alpha=alpha, method='fdr_bh')
        results.loc[valid_pvals, 'am_rate_q_value'] = q_values
        results['am_rate_significant'] = results['am_rate_q_value'] < alpha
    else:
        results['am_rate_q_value'] = np.nan
        results['am_rate_significant'] = False

    return results


In [ ]:
# import numpy as np
# import pandas as pd
# from scipy import stats
# from statsmodels.stats.multitest import multipletests

# def test_am_rate_ratio(gene_df, alpha=0.1):
#     """
#     For each gene, compare
#         rate_syn = n_syn / exp_syn_counts
#         rate_am  = observed_mis_am_score / exp_mis_scores
#     via a one-sided Poisson LRT (H0: rate_am <= rate_syn vs. H1: rate_am > rate_syn).
#     """
#     results = gene_df.copy()
#     p_vals = []

#     for _, row in results.iterrows():
#         n_syn   = row['n_syn']   # Add small pseudo count
#         e_syn   = row['exp_syn_counts'] + 1e-8  # Add small pseudo count
#         n_am    = row['observed_mis_am_score']
#         e_am    = row['exp_mis_scores']

#         # require all positive
#         if all(pd.notnull([n_syn,e_syn,n_am,e_am])) and e_syn>0 and e_am>0:
#             # trivial case: observed rate <= expected
#             if (n_am/e_am) <= (n_syn/e_syn):
#                 p = 1.0
#             else:
#                 # null: single common rate mu
#                 mu0 = (n_syn + n_am) / (e_syn + e_am)
#                 ll0 = stats.poisson.logpmf(n_syn, mu0 * e_syn) \
#                     + stats.poisson.logpmf(n_am , mu0 * e_am)

#                 # alt: separate rates
#                 mu_syn = n_syn / e_syn
#                 mu_am  = n_am  / e_am
#                 ll1 = stats.poisson.logpmf(n_syn, mu_syn * e_syn) \
#                     + stats.poisson.logpmf(n_am , mu_am  * e_am)

#                 lr = -2 * (ll0 - ll1)
#                 p  = 0.5 * stats.chi2.sf(lr, df=1)  # one‐sided
#         else:
#             p = np.nan

#         p_vals.append(p)

#     results['am_rate_p_value'] = p_vals

#     # FDR correction
#     mask = results['am_rate_p_value'].notna()
#     if mask.any():
#         _, qv, _, _ = multipletests(results.loc[mask,'am_rate_p_value'],
#                                      alpha=alpha, method='fdr_bh')
#         results.loc[mask, 'am_rate_q_value'] = qv
#         results['am_rate_significant'] = results['am_rate_q_value'] < alpha
#     else:
#         results['am_rate_q_value'] = np.nan
#         results['am_rate_significant'] = False

#     return results


In [24]:
# Set significance threshold and add significance column
gene_results = test_am_rate_ratio(gene_results, alpha=0.1)
FDR_THRESHOLD = 0.05
gene_results['am_significant'] = gene_results['am_rate_q_value'] < FDR_THRESHOLD

# Show summary of significant genes
sig_count = gene_results['am_significant'].sum()
print(f"Found {sig_count} genes with significant AlphaMissense score differences (q < {FDR_THRESHOLD})")

Found 363 genes with significant AlphaMissense score differences (q < 0.05)


In [16]:
output_file = "/Volumes/T7_Shield/Lab/T7_UCSD/github_repos/BENG285_team3/project_2/alphamissense/gene_results_with_am_significance.csv"
gene_results.to_csv(output_file, index=False)
print(f"Saved updated gene results with AM significance to {output_file}")

Saved updated gene results with AM significance to /Volumes/T7_Shield/Lab/T7_UCSD/github_repos/BENG285_team3/project_2/alphamissense/gene_results_with_am_significance.csv


In [ ]:
# gene_results = pd.read_csv('/Volumes/T7_Shield/Lab/T7_UCSD/github_repos/BENG285_team3/project_2/alphamissense/gene_results_with_am_significance.csv')
groud_truth = pd.read_csv("/Users/wengjiaming/Documents/UCSD/2025_spring/BENG285/IntOGen-DriverGenes_LUAD.tsv", sep="\t")
groud_truth = groud_truth[groud_truth['Symbol'].notna()]

# Compare the significant genes with the ground truth
def compare_significant_genes(gene_results, ground_truth):
    """
    Compare significant genes with the ground truth.
    
    Parameters:
    -----------
    gene_results: pd.DataFrame
        DataFrame containing gene results with significance.
    ground_truth: pd.Series
        Series containing ground truth gene names.
        
    Returns:
    --------
    None
    """
    significant_genes = gene_results[gene_results['am_significant'] == True]['gene_name']
    
    # Find true positives and false positives
    true_positives = set(significant_genes) & set(ground_truth)
    false_positives = set(significant_genes) - set(ground_truth)
    
    print(f"True Positives: {len(true_positives)}")
    print(f"False Positives: {len(false_positives)}")
    
compare_significant_genes(gene_results, groud_truth['Symbol'])

True Positives: 17
False Positives: 346
